# PART I 
>>>Using Pipeline preprocess the shopping data, do test and train and then test using test data set

PROBLEM IDENTIFICATION: The given dataset is on clickstream from an online store offering clothing for pregnant women contains valuable information that can be utilized for various purposes, such as understanding user behavior, optimizing webpage layout, and predicting product categories. The problem identified here is to predict the product category based on the given features in the dataset. This prediction can be useful for targeted marketing, personalized recommendations, and improving the overall user experience on the online store.

OUR TARGET VARIABLE IS : PAGE
For a certain input we can find the page number of the product category we are trying to search in the e-store website.

In [15]:
#Reference: https://www.kaggle.com/cemutku/k-means-and-hierarchical-clustering-implementation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
#reading the shopping dataset

excel_file_path = "C://Users//HP//Downloads//EXCEL DATA.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)


In [19]:
df.head()

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


In [20]:
## A. Data manipulation
import numpy as np
import pandas as pd

# 1.2 for data splitting
from sklearn.model_selection import train_test_split

## B. Transformers for predictors:

# 1.3 Class for imputing missing values
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
from sklearn.impute import SimpleImputer

# 1.4 One hot encode categorical data--Convert to dummy
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
from sklearn.preprocessing import OneHotEncoder as onehot

# 1.5 Scale numeric data
from sklearn.preprocessing import StandardScaler

## C. Transformer for target:

# 1.6 Label encode target column
from sklearn.preprocessing import LabelEncoder


## D. Composite Transformers:

# 1.7 Class for applying multiple data transformation
#     jobs parallely
from sklearn.compose import ColumnTransformer

# 1.8 Pipeline class: Class for applying multiple
#     data transformations sequentially
from sklearn.pipeline import Pipeline

## E. Estimator

#random forest
from sklearn.ensemble import RandomForestClassifier

# 1.10 To plot pipeline diagram
from sklearn import set_config

In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
# 2.0 Import warnings module
import warnings
# 2.1 Do not print warnings on screen
warnings.filterwarnings("ignore")

Read and preprocess data
(#3.0--#3.2)
Read data, create simple cat features and shuffle data

In [23]:
print (df.shape)

(65535, 14)


In [24]:
df=df.drop(['session ID'],axis=1)

In [25]:
df.isna().sum()

year                       0
month                      0
day                        0
order                      0
country                    0
page 1 (main category)     0
page 2 (clothing model)    0
colour                     0
location                   0
model photography          0
price                      0
price 2                    0
page                       0
dtype: int64

In [26]:
# 3.2 Randomly shuffle data as values 
#     in 'credibility' column have an order
df = df.sample(frac = 1) 
print (df)
print (df.shape)    

       year  month  day  order  country  page 1 (main category)  \
18801  2008      4   10     19       29                       4   
16311  2008      4    8     71       24                       4   
25625  2008      4   14     11       29                       3   
40681  2008      4   24      5        9                       2   
22849  2008      4   12     12        9                       3   
...     ...    ...  ...    ...      ...                     ...   
1524   2008      4    1     18       29                       2   
17919  2008      4    9      6       29                       3   
40673  2008      4   24      8       29                       4   
49315  2008      5    1     16       29                       4   
5756   2008      4    2     25       29                       1   

      page 2 (clothing model)  colour  location  model photography  price  \
18801                     P59       2         2                  2     38   
16311                      P9       4    

Data splitting
(#3.3--#4.1)
i) Separate predictors and target.
ii) Create train/test data.

In [27]:
# 3.3    Popout target
#        to separate predictors and target

y = df.pop('page')
y[:3]      # Pandas Series

# 3.4   Create an alias of predictors dataset 
X = df     # X is another name for df
X.shape    # (14,7)

18801    4
16311    1
25625    4
Name: page, dtype: int64

(65535, 12)

In [28]:
# 4.0 Split dataset. We will preprocess X_train and apply that
#     processing to X_test later
X_train,X_test, y_train, y_test = train_test_split(
                                                    X,                   # Data features
                                                    y,                   # Target column
                                                    test_size = 0.2      # split-ratio
                                                    )

# 4.1 Note the use of f-string for printing
f"X_train shape: {X_train.shape}"    # (11,7)
print()
f"X_test.shape : {X_test.shape}"     # (3,7)
print()
f"y_train shape: {y_train.shape}"    # (11,)
print()
f"y_test shape : {y_test.shape}"     # (3,)

'X_train shape: (52428, 12)'

'X_test.shape : (13107, 12)'

'y_train shape: (52428,)'

'y_test shape : (13107,)'

### Separate out categorical and numerical features
Any (integer) feature with less than 5 unique-values is categorical. Here is how we process:

I. Get a list of `cat_cols` and `num_cols`.

II. Categorical features processing:
-  Imputation 
> - Impute most-frequent value
> - Impute constant value
- Encoding
> -  Target encoding of target only
> -  OneHotEncoding of rest features

III. Numeric features processing
-  Imputation 
> - Impute median
> - Impute mean
- Scaling
> -  Min-Max or StandardScaling


In [29]:
### 4.3
###    Which columns are categorical
###    but disguised as integers

# 4.3 How many unique values per column.
#     Check every column
#     We will assume that if number of unique values
#      are 4 or less it is categorical column else numeric

print()
X_train.nunique()        # Total no of unique values in each column

# 4.4 If no. of unique values less than 5, it is categorical
print("\n------\n")
print()
X_train.nunique() < 5    # All True are categorical

year                         1
month                        2
day                         30
order                      154
country                     36
page 1 (main category)       4
page 2 (clothing model)    217
colour                      14
location                     6
model photography            2
price                       20
price 2                      2
dtype: int64


------




year                        True
month                       True
day                        False
order                      False
country                    False
page 1 (main category)      True
page 2 (clothing model)    False
colour                     False
location                   False
model photography           True
price                      False
price 2                     True
dtype: bool

In [30]:
# 4.5 Extract list of cat_cols and num_cols:

# 4.6 First note which are cat and which are num
dg = (X_train.nunique() < 5)  

# 4.7 Then filter out names from Series index 
cat_cols = dg[dg==True].index.tolist()
num_cols = dg[dg==False].index.tolist()

In [31]:
# 4.8 Here are the columns
cat_cols    #  4
print()
num_cols    #  3

['year', 'month', 'page 1 (main category)', 'model photography', 'price 2']

['day',
 'order',
 'country',
 'page 2 (clothing model)',
 'colour',
 'location',
 'price']

In [32]:
num_cols_mean=['month','day',
 'order',
 'country',
 'colour',
 'location',
 'price']

cat_cols_mf       = ['year', 'page 1 (main category)', 'page 2 (clothing model)', 'model photography', 'price 2'] 

In [33]:
# 4.11 So we have 2 datasets for imputing.
#      Have a look:
X_train[num_cols_mean]              # Num dataset, impute by 'mean'   strategy
print()
X_train[cat_cols_mf]                # Cat dataset, impute by 'most_frequent' strategy
print()


,month,day,order,country,colour,location,price
56140,5,8,6,29,7,6,33
17367,4,9,6,29,9,5,23
46248,4,29,21,29,13,1,48
9554,4,4,1,29,2,6,38
44567,4,28,5,29,6,3,38
...,...,...,...,...,...,...,...
50409,5,3,23,29,14,4,43
62664,5,14,12,9,4,2,62
1075,4,1,13,29,2,5,62
17053,4,9,5,29,2,6,48


,year,page 1 (main category),page 2 (clothing model),model photography,price 2
56140,2008,4,P16,1,2
17367,2008,3,C14,1,2
46248,2008,3,C55,1,1
9554,2008,4,P17,2,1
44567,2008,2,B25,1,2
...,...,...,...,...,...
50409,2008,3,C12,1,1
62664,2008,1,A22,1,1
1075,2008,2,B14,1,1
17053,2008,2,B34,1,2


In [34]:
X_train_c = X_train.copy()
X_test_c  = X_test.copy()

In [35]:
# 10.1 Instantiate Pipeline object for processing
#      cat data. Impute = most_frequent
pipe1  =  Pipeline(
                     [
                      ('mf', SimpleImputer(strategy='most_frequent')),
                      ('ohe', onehot())
                     ]
                   )

In [36]:
# 10.4 Instantiate Pipeline object for 
#      processing numerical data. Impute = mean
#      Pipeline as a composite transformer
pipe2 =     Pipeline(
                      [
                       ('si', SimpleImputer(strategy='mean')),
                       ('ss1', StandardScaler())
                      ]
                    )

In [37]:
# 10.5
# Test transformers:
# Feed data to each pipe to see if it is working
# It is like testing a sub-component
# before full-plumbing is done.
# No error should come.
pipe1.fit_transform(X_train[cat_cols_mf])
pipe2.fit_transform(X_train[num_cols_mean])

<52428x226 sparse matrix of type '<class 'numpy.float64'>'
	with 262140 stored elements in Compressed Sparse Row format>

array([[ 1.66813788, -0.52546607, -0.30496052, ...,  0.16653361,
         1.58880086, -0.85132845],
       [-0.59947083, -0.40550781, -0.30496052, ...,  0.63555266,
         1.00452655, -1.65684531],
       [-0.59947083,  1.99365749,  0.90259137, ...,  1.57359076,
        -1.3325707 ,  0.35694685],
       ...,
       [-0.59947083, -1.36517393,  0.2585637 , ..., -1.00601401,
         1.00452655,  1.48467046],
       [-0.59947083, -0.40550781, -0.38546398, ..., -1.00601401,
         1.58880086,  0.35694685],
       [-0.59947083,  0.79407484,  0.01705332, ...,  0.16653361,
         0.42025224, -0.85132845]])

In [38]:
# 10.6 Collecting all pipes in column transformer
#     along with column names
#     All pipes operate parallely
#                       some-name  transformer     col-names

ct_transformer = ColumnTransformer(
                                    [
                                      ('p_mf',   pipe1,   cat_cols_mf),
                                      ('p_mean', pipe2,   num_cols_mean),
                                    ]
                                  )

In [39]:
# 10.7 Test column transformet
ct_transformer.fit_transform(X_train)

<52428x233 sparse matrix of type '<class 'numpy.float64'>'
	with 629136 stored elements in Compressed Sparse Row format>

In [40]:
# 10.8 Final pipeline for transformation and modeling
#     final_pipe is both a 
outer_pipe = Pipeline(
                      [
                       ('ct', ct_transformer),        # Column transformer object
                       ('rf', RandomForestClassifier()) # Estimator
                      ]
                     )

In [41]:
# 11.0 Train on data using final_pipe
#     We use (X_train, y_train)

outer_pipe.fit(X_train,y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('p_mf',
                                                  Pipeline(steps=[('mf',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder())]),
                                                  ['year',
                                                   'page 1 (main category)',
                                                   'page 2 (clothing model)',
                                                   'model photography',
                                                   'price 2']),
                                                 ('p_mean',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer()),
                                                                  ('ss1',
                                                                   StandardScaler())]),
                                                  ['month', 'day', 'order',
                                                   'country', 'colour',
                                                   'location', 'price'])])),
                ('rf', RandomForestClassifier())])

In [42]:
# 11.1 Make prediction on test data
#     Note that there is no need to separately
#     transform X_test. Pipes take care of that

outer_pipe.predict(X_test)

array([4, 2, 1, ..., 1, 1, 1], dtype=int64)

In [43]:
# 11.2 But what is the actual y_test
y_test
# le.transform(y_test)

20068    4
63397    2
47034    1
8736     1
21465    1
        ..
45750    2
46542    1
22186    1
25674    1
39751    1
Name: page, Length: 13107, dtype: int64

In [44]:
# 11.3 Display pipeline as diagram
# Ref:
#  https://scikit-learn.org/stable/modules/compose.html#visualizing-composite-estimators

from sklearn import set_config
set_config(display='diagram')
outer_pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('p_mf',
                                                  Pipeline(steps=[('mf',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder())]),
                                                  ['year',
                                                   'page 1 (main category)',
                                                   'page 2 (clothing model)',
                                                   'model photography',
                                                   'price 2']),
                                                 ('p_mean',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer()),
                                                                  ('ss1',
                                                                   StandardScaler())]),
                                                  ['month', 'day', 'order',
                                                   'country', 'colour',
                                                   'location', 'price'])])),
                ('rf', RandomForestClassifier())])

In [45]:
# 11.4 Pipeline as text
set_config(display='text')
#final_pipe_transformer_estimator
outer_pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('p_mf',
                                                  Pipeline(steps=[('mf',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder())]),
                                                  ['year',
                                                   'page 1 (main category)',
                                                   'page 2 (clothing model)',
                                                   'model photography',
                                                   'price 2']),
                                                 ('p_mean',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer()),
        

# Testing with test data 

In [51]:
df.head()

,year,month,day,order,country,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2
18801,2008,4,10,19,29,4,P59,2,2,2,38,1
16311,2008,4,8,71,24,4,P9,4,3,2,23,2
25625,2008,4,14,11,29,3,C57,5,1,2,33,2
40681,2008,4,24,5,9,2,B33,6,5,2,33,2
22849,2008,4,12,12,9,3,C9,6,3,1,48,1


In [46]:
df_test = pd.read_csv("C://Users//HP//Downloads//TestData.csv")

In [47]:
df_test


,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2
0,2008,4,1,3,29,2,2,B24,11,2,1,57,1


In [48]:
# 11.1 Make prediction on test data
#     Note that there is no need to separately
#     transform X_test. Pipes take care of that

outer_pipe.predict(df_test)

array([2], dtype=int64)

OBSERVATION: The model predicts a class or label for the given test data, and in this case, it predicts the class label 2.The prediction result is an array with a single element which [2] as we have one taget variable that is Page which means  it belongs to the Page category 2 ( In Page Number 2 within the e-store website we can find the test data that we want to search).  